# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Frontend
💩

# Coordle Backend

In [123]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain

import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, logs_filename: str):
        self.output_dir = output_dir
        self.prefix = prefix
        self.logs_filename = logs_filename
        self.epoch = 0

    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        if self.epoch == 0:
            loss = cum_loss
        else:
            loss = cum_loss - self.loss_previous_step
        self.loss_previous_step = loss
        with open(join_path(self.output_dir, self.logs_filename), 'a+') as file:
            file.write(f'Epoch #{self.epoch}, loss: {loss}\n')
        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, start_epoch: int = 1):
        self.output_dir = output_dir
        self.prefix = prefix
        self.epoch = start_epoch

    def on_epoch_end(self, model):        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

In [4]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [247]:
# %%time
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index

def test_SentVectorDoc():
    docsample = df[df.cord_uid == 'vs5yondw']
#     doc = SentVectorDoc(docsample['cord_uid'], docsample['title'])
#     doc, _ = doc.fit(docsample['body_text'])
    gaga = Index()
    gaga.build_from_df(
        docsample,
#         df[:32],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=False,
        workers=-1
    )
    
    return gaga
#     return doc

def test_Index():
    coordle = Index()
    coordle.build_from_df(
        df.iloc[:256],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=True,
        workers=-1
    )
    return coordle

index = test_Index()
# fuck = test_SentVectorDoc()

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 256/256 [00:00<00:00, 401.60it/s]


In [30]:
# save_pickle(index, 'coordle/coordle_index_8k_0904.p')

In [51]:
query = 'retarded white woman'
docs1, score = index.search(query, verbose=True)
print(docs1)
print()
docs2 = index.advanced_search(query, verbose=True)
print(docs2)

Query tokens:  ['retarded', 'white', 'woman']
[8bnny6hj, zp9k1k3z, 5xswwney, fdtwagr1, hnl4a33n, 1rzbaj02, 15mv5yx7, 2ole6ya7, vs5yondw, j1iylpgm, ltf0xacf, eyk015n3, bgygebgx, mnhg329v, y2ruobm0, ak7bf0gq, uq88w4fm, i94lyfsh, lno01041, y2uhnlpd, ensep6lk, 9jpc42h1, tjzlssal, 34jn37sb, p8bzkmas, 67kyfybe, rwlryr0h, y3hgnb6u, ootj52fs, vdk2pxkk, 0b1qaecu, 9hru9fyg, vo2oe2wr, piwdsazx, uhrijlyz, mj9ea464, i6vuhaiv, fu0x6bqb, qg2pb884, ret6iq3o, 7isxu2jm, fvrhzc7x, pq0atbfg, x4fbph3d, 8ext7xa7, dksh41wz, mvv1rg68, tmfrc2e2, x8uzlsn7, sb43y1cl, pbhdzshl, 7lzyvzg2, gu0930s3, qwh8ei60, 5yrouyvo, h1gnp62r, 5abus746, 8rs8ilio, 8kooncil, oir3rlb7, 5lmrthmb, wwf90zxt]

Query tokens: ['retarded', 'white', 'woman']
deque(['retarded', 'white', 'woman'])
{5lmrthmb, wwf90zxt, zp9k1k3z}


In [149]:
query = 'retarded    (white AND woman)'
querytokens = re.split('([^a-zA-Z0-9])', query)
# Gotta do this to capture parenthesis
querytokens = chain.from_iterable([t.split() for t in querytokens])

print(list(querytokens))

['retarded', '(', 'white', 'AND', 'woman', ')']


In [190]:
A = deque([1,2,3])
A.insert(1,10)
A

deque([1, 10, 2, 3])

In [194]:
print('\\\\')

\\


In [219]:
'(' in '()'

True

In [261]:
def get_logical_querytokens(query: str, errmsgs: list):
    querytokens = re.split('([^a-zA-Z0-9])', query)
    # Gotta do this to capture parenthesis
    querytokens = chain.from_iterable([t.split() for t in querytokens])

    q1 = deque(querytokens)
    q2 = deque()

    operators = {'OR','AND','NOT'}    
    q2.append(q1.popleft())

    p_counter = 0
    while len(q1) > 0:
        token = q1.popleft()
        
        if q2[-1] == '(':
            q2.append(token)
            p_counter += 1
            continue
            
        if token == ')':
            q2.append(token)
            p_counter -= 1
            continue
        
        # If preceeding token was operator
        if q2[-1] in operators:
            q2.append(token)
        # If preceeding token was not operator
        else:
            # If current token is not an operator
            if token not in operators:
                q2.append('OR')
            q2.append(token)
    return q2

def assert_query(querytokens: deque, errmsgs: list) -> bool:
    '''
    Check if query is properly formatted. Returns True is everthing is ok,
    else False. 
    '''
    q = querytokens.copy()
    operators = {'OR','AND','NOT'}
    flag = True
    curr = q.popleft()
    # If query is starting with an operator
    if curr in operators:
        errmsgs.append(f'SyntaxError: First token "{curr}" is an operator')
        flag = False
    
    # Stray closing parenthesis
    if curr == ')':
        errmsgs.append(f'SyntaxError: First token is stray closing parenthesis')
        flag = False
    
    # Handle case for 1 token
    if len(q) == 0:
        return flag
    
    prev = curr
    # If more than one token left
    while len(q) > 1:
        curr = q.popleft()
        # If curr is operator
        if curr in operators:
            # Two succeeding operators
            if prev in operators:
                errmsgs.append(f'SyntaxError: Two succeeding operators "{prev} {curr}"')
                flag = False
        prev = curr
    
    # Should only be one token left when interpreter is here
    
    # If ending with an operator
    if q[0] in operators:
        errmsgs.append(f'SyntaxError: Last token "{q[0]}" is an operator')
        flag = False
        
    return flag

queries = [
    'retarded',
    'retarded white woman',
    'retarded OR white OR woman',
    'retarded white AND woman',
    'retarded OR white NOT woman',
    'retarded (white NOT woman)',
    'retarded (white NOT woman)',
    'OR retarded AND white woman',
    'retarded AND AND white NOT woman',
    'retarded (white NOT woman) AND',
    ')retarded ((white NOT woman) AND',
    'retarded ((white NOT woman)',
]
    
for query in queries:
    errmsgs=[]
    q = get_logical_querytokens(query, errmsgs)
    print(str(q) +' '+ str(assert_query(q, errmsgs)))
    if errmsgs: print(errmsgs)
    
    print()

deque(['retarded']) True

deque(['retarded', 'OR', 'white', 'OR', 'woman']) True

deque(['retarded', 'OR', 'white', 'OR', 'woman']) True

deque(['retarded', 'OR', 'white', 'AND', 'woman']) True

deque(['retarded', 'OR', 'white', 'NOT', 'woman']) True

deque(['retarded', 'OR', '(', 'white', 'NOT', 'woman', ')']) True

deque(['retarded', 'OR', '(', 'white', 'NOT', 'woman', ')']) True

deque(['OR', 'retarded', 'AND', 'white', 'OR', 'woman']) False
['SyntaxError: First token "OR" is an operator']

deque(['retarded', 'AND', 'AND', 'white', 'NOT', 'woman']) False
['SyntaxError: Two succeeding operators "AND AND"']

deque(['retarded', 'OR', '(', 'white', 'NOT', 'woman', ')', 'AND']) False
['SyntaxError: Last token "AND" is an operator']

deque([')', 'OR', 'retarded', 'OR', '(', '(', 'white', 'NOT', 'woman', ')', 'AND']) False
['SyntaxError: First token is stray closing parenthesis', 'SyntaxError: Last token "AND" is an operator']

deque(['retarded', 'OR', '(', '(', 'white', 'NOT', 'woman', ')

In [257]:
print(f'Cardinality 1: {len(docs1)}')
print(f'Cardinality 2: {len(docs2)}')

docs1_list = sorted(docs1, key=lambda x: x.uid)
docs2_list = sorted(docs2, key=lambda x: x.uid)

# Set equal gave unexpceted results
print(f'Equal?: {all([d1 == d2 for d1, d2 in zip(docs1_list, docs2_list)])}')

Cardinality 1: 62
Cardinality 2: 3
Equal?: False


In [8]:
with open('textfile.txt', 'w+') as f:
#     uid = docs2_list[0].uid
#     uid = 'dlh93ax6'
#     uid = 'zp9k1k3z'
#     uid = 'vs5yondw'
    uid = 'i94lyfsh'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping i94lyfsh


In [258]:
docs, scores = index.search('retarded white woman', verbose=True)
# docs, scores = index.search('!', verbose=True)
n = 16
print()
for doc, score in zip(docs[:n], scores[:n]):
    print(f'{doc.uid}   {str(doc.title)[:69]:<69}   {score:.4f}')

Query tokens:  ['retarded', 'white', 'woman']

8bnny6hj   Regional localization of virus in the central nervous system of mice    7.4594
zp9k1k3z   Clinical and immunologic studies in identical twins discordant for sy   7.4227
5xswwney   Interferon induction in swine lymphocyte antigen-defined miniature pi   5.0407
fdtwagr1   Intracisternal virus-like particles in brain of a multiple sclerosis    4.5837
hnl4a33n   Expression of Ia antigen on perivascular and microglial cells after s   4.1718
1rzbaj02   Leptomeningitis and polycaryocyte formation in the CNS of rats inocul   4.0657
15mv5yx7   Characterization of coronavirus JHM variants isolated from wistar fur   3.7061
2ole6ya7   The pathogenesis of multiple sclerosis Additional considerations        3.2869
vs5yondw   Acute bronchitis: Results of U.S. and European trials of antibiotic t   3.0751
j1iylpgm   Characterization of murine hepatitis virus (JHM) RNA from rats with e   2.7213
ltf0xacf   Paediatric gastroenteritis in the eastern 

In [ ]:
raise ValueError

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index, AI_Index

def most_similar(token):
    if token in model.wv:
        return model.wv.most_similar(token)
    return []

def test_AI_Index(model):
    coordle = AI_Index(most_similar, 3)
    coordle.build_from_df(
        df.iloc[:1024],
        'cord_uid',
        'title',
        'body_text',
        use_multiprocessing=True,
        workers=-1,
        verbose=True
    )
    return coordle

ai_index = test_AI_Index(model)
            

In [ ]:
docs, scores = ai_index.search('retarded white woman', verbose=True)
n = 16
print()
for doc, score in zip(docs[:n], scores[:n]):
    print(f'{doc.uid}   {doc.title[:80]:<80}   {score:.4f}')

## Multiprocessing showdown

In [ ]:
from multiprocessing import Pool
from time import time

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text), position=0, total=len(df)))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text), position=0, total=len(df))
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text[:2048]), position=0, total=2048))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text[:2048]), position=0, total=2048)
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')